In [1]:
import pandas as pd
import numpy as np

# Basic Static Plot

In [16]:
nr_models = 10
short_controls = ['age', 'grade', 'iq_score', 'gender']

df1 = pd.DataFrame()

for i in range(nr_models):
    ctrl_names = short_controls
    factor = 0.5
    nr_params = len(ctrl_names)
    df = pd.DataFrame()
    df['param_name'] = ctrl_names
    df['param_value'] = factor * np.arange(nr_params) + np.random.normal(0, 1, nr_params)
    df['model'] = 'model_' + str(i)
    
    df1 = pd.concat([df1, df], axis=0, sort=False)
    
df1.reset_index(inplace=True, drop=True)
df1.head()

,param_name,param_value,model
0,age,1.025307,model_0
1,grade,1.538086,model_0
2,iq_score,1.655725,model_0
3,gender,1.360541,model_0
4,age,-3.387153,model_1


In [36]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show, output_notebook

def static_comparison_plot(df):
    source = ColumnDataSource(df)
    
    # create the "canvas"
    plot = figure(
        title="Comparison Plot",
        y_range=df["param_name"].unique())
    
    # add circles representing the parameter value
    circle_glyph = plot.circle(
        source=source,
        x="param_value",
        y="param_name",
        size=12)
    show(plot)

In [37]:
# display the plot in the notebook 
# same as %matplotlib inline
output_notebook() 

static_comparison_plot(df1)

Loading BokehJS ...

This already supports some basic interactivity out of the box:
- zooming
- moving

# Display Additional Information when Hovering

In [130]:
from bokeh.models import HoverTool


def comparison_plot_with_hovering(df):
    source = ColumnDataSource(df)
    
    # create the "canvas"
    plot = figure(
        title="Comparison Plot",
        y_range=df["param_name"].unique())
    
    # add circles representing the parameter value
    circle_glyph = plot.circle(
        source=source,
        x="param_value",
        y="param_name",
        size=12)
    
    # HoverTool
    tooltips = [
        ("parameter value", "@param_value"),
        ("model", "@model")]
    hover = HoverTool(renderers=[circle_glyph], tooltips=tooltips)
    plot.tools.append(hover)

    show(plot)

In [131]:
comparison_plot_with_hovering(df1)

# Select points of the same model when one estimate is selected

In [47]:
# two components of Bokeh
# need custom callback when point is tapped
# server OR JavaScript
# ...

In [235]:
%%javascript

function select_model_points(source){

// adapted from https://stackoverflow.com/a/44996422
var chosen = source.selected.indices;
if (typeof(chosen) == "number"){
    var chosen = [chosen]
};

var chosen_models = []; 

for (var i = 0; i < chosen.length; ++ i){
    chosen_models.push(source.data['model'][chosen[i]])
};
        
var chosen_models_indices = []; 

for (var i = 0; i < source.data['index'].length; ++ i){
    if (chosen_models.includes(source.data['model'][i])){
        chosen_models_indices.push(i)
    };
};

source.selected.indices = chosen_models_indices 
source.change.emit();
    
}

<IPython.core.display.Javascript object>

In [239]:
with open('callback.js', 'r') as f:
    js_code = f.read()

In [245]:
from bokeh.models import CustomJS

def comparison_plot_with_model_selection(df, js_code=js_code):
    # safety measure to make sure the index 
    # gives the position in the arrays 
    # that the source data dictionary points to
    safe_df = df.reset_index(drop=True)
    
    source = ColumnDataSource(df)
    
    # create the "canvas"
    plot = figure(
        title="Comparison Plot",
        y_range=df["param_name"].unique())
    
    # add circles representing the parameter value
    circle_glyph = plot.circle(
        source=source,
        x="param_value",
        y="param_name",
        size=12)
    
    # HoverTool
    tooltips = [
        ("parameter value", "@param_value"),
        ("model", "@model")]
    hover = HoverTool(renderers=[circle_glyph], tooltips=tooltips)
    plot.tools.append(hover)
    
    # TapTool
    js_kwargs = {"source": source}
    js_callback = CustomJS(args=js_kwargs, code=js_code)
    tap = TapTool(callback=js_callback)
    plot.tools.append(tap)

    show(plot)

In [246]:
comparison_plot_with_model_selection(df1)

In [255]:
# add some styling

def styled_comparison_plot(df, js_code=js_code):
    # safety measure to make sure the index 
    # gives the position in the arrays 
    # that the source data dictionary points to
    safe_df = df.reset_index(drop=True)
    
    source = ColumnDataSource(df)
    
    # create the "canvas"
    plot = figure(
        title="Comparison Plot",
        y_range=df["param_name"].unique(),
        
        toolbar_location="right", 
        plot_width=600,
        plot_height=300, 
        tools="reset,save",
    )
        
    # add circles representing the parameter value
    circle_glyph = plot.circle(
        source=source,
        x="param_value",
        y="param_name",
        size=12,
        
        fill_color="#035096",
        selection_fill_color="firebrick",
        nonselection_fill_alpha=0.2,
    )
    
    # HoverTool
    tooltips = [
        ("parameter value", "@param_value"),
        ("model", "@model")]
    hover = HoverTool(renderers=[circle_glyph], tooltips=tooltips)
    plot.tools.append(hover)
    
    # TapTool
    js_kwargs = {"source": source}
    js_callback = CustomJS(args=js_kwargs, code=js_code)
    tap = TapTool(callback=js_callback)
    plot.tools.append(tap)

    show(plot)

In [256]:
styled_comparison_plot(df1)

# Add groups of parameters to be plotted into separate plots

# Add confidence intervals as whiskers that are only displayed when the model is selected

# Add interactive legend